In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 2.x

Mounted at /content/drive


In [2]:
!pip install tiffile


!pip install gputools

!pip install imagecodecs
!pip install vollseg


     |████████████████████████████████| 153kB 2.9MB/s 
     |████████████████████████████████| 194kB 13.8MB/s 
     |████████████████████████████████| 880kB 34.9MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
  Created wheel for reikna: filename=reikna-0.7.5-cp37-none-any.whl size=122266 sha256=cbdf9825d64373755d59308385e837f95b165fd619fb3dd1321d3b2ead25f68e
  Stored in directory: /root/.cache/pip/wheels/82/2d/ba/12c9ba3637183463c471bcf352f5bc1703ab7dfbec9842f04a
  Created wheel for pytools: filename=pytools-2021.2.6-py2.py3-none-any.whl size=60643 sha256=1b6900a79b513100195adf31ae727d1e8792656c19177ef54e571f5bb86ae1bf
  Stored in directory: /root/.cache/pip/wheels/8c/a6/65/447b9b4fd1d9bde84ad2fea2431a38f69f3fb573476a98ae03
Successfully built reikna pytools
     |████████████████████████████████| 30.0MB 172kB/s 
     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 2.4MB 10.2MB/s

In [3]:
%cd '/content/drive/My Drive/VollSeg/'
import os
import glob
import sys
import numpy as np
from tqdm import tqdm
from tifffile import imread, imwrite
from pathlib import Path
from vollseg import SmartSeeds3D



/content/drive/My Drive/VollSeg


# In the cell below specify the following:

1) Directory where the training data is, inside this directory there should be the two subfolders called Raw and Mask. Inside the Raw folder are the raw images and inside the Mask folder are the labelled images.

2) The training data for doing UNET training is stored in NPZ format so please specify the NPZ filename which is suitable for your data.

3) Model directory is where the trained Neural network models are stored, please chooose a location if you want to change the default location which is where the training data is.

4) Copy Model name is optional, in case you have a previouis trained model and want to re-train it on new data but store it with a new name.

5) Model name is the unique name of the trained models.

In [4]:
Data_dir = '/content/drive/My Drive/data/'
NPZ_filename = 'VolumeSeg'
Model_dir = '/content/drive/My Drive/data/'
Model_Name = 'VolumeSeg'

# In this cell choose the network training parameters for the Neural Network

1) NetworkDepth = Depth of the network, with each increasing depth the image is downsampled by 2 hence the XYZ dimension of the data / 2^depth has to be greater than 1.

2) Epochs, training for longer epochs ensures a well converged network and requires longer GPU runtimes.

3) Learning rate is the parameter which controls the step size used in the optimization process and it should not be greater than 0.001 at the start of the training.

4) batch size controls the number of images used for doing stochastic gradient descent and is a parameter that is limited by the GPU RAM available, if you do not have a lot of ran batch size < 10 should be optimal. 

5) PatchX,Y,Z is the patch size used for making patches out of the iamge data. The original image is broken down into patches for training. Patch size is chosen based on having enough context for the network to learn but at the same time not being too big to obscure memory usage.

6) Kernel is the receptive field of the neural network, usual choices are 3,5 or 7 but not larger than that. This is the size of the convolutional kernel used in the network

7) n_patches_per_image is the number of patches sampled for each image to create the npz file, choose an optimal value so that the file is not too big for the computer memory. 

8) Rays is the number of rays used the learn the distance map, low rays decreases the spatial resoultion and high rays are able to resolve the shape better.


9) OpenCL is a boolean parameter that is set true if you want to do some opencl computations on the GPU, this requires GPU tools but if you do not have them set this to false.

Some optimal values have been chosen by default and should work well for any NVDIA enabled GPU computer

In [5]:
#Network training parameters
NetworkDepth = 3
Epochs = 100
LearningRate = 1.0E-4
batch_size = 1
PatchX = 128
PatchY = 128
PatchZ = 16
Kernel = 3
n_patches_per_image = 16
Rays = 128
startfilter = 48
use_gpu_opencl = True
GenerateNPZ = True
TrainUNET = False
TrainSTAR = False

# Generate the npz file first and then train the model

In [6]:

SmartSeeds3D(BaseDir = Data_dir, NPZfilename = NPZ_filename, model_name = Model_Name, model_dir = Model_dir, n_patches_per_image = n_patches_per_image,GenerateNPZ = GenerateNPZ, TrainUNET = TrainUNET, TrainSTAR = TrainSTAR, PatchX= PatchX, PatchY= PatchY, PatchZ = PatchZ,  use_gpu = use_gpu_opencl,  batch_size = batch_size, depth = NetworkDepth, kern_size = Kernel, startfilter = startfilter, n_rays = Rays, epochs = Epochs, learning_rate = LearningRate)

Instance segmentation masks: 24
Semantic segmentation masks: 24
   24 raw images x    1 transformations   =    24 images
   24 images     x   16 patches per image =   384 patches in total
Input data:
/content/drive/My Drive/data/: target='BinaryMask/', sources=['Raw/'], axes='ZYX', pattern='*.tif*'
Transformations:
1 x Identity
Patch size:
16 x 128 x 128


100%|██████████| 24/24 [01:11<00:00,  2.96s/it]


Saving data to /content/drive/My Drive/data/VolumeSeg.npz.


In [ ]:

TrainUNET = True
TrainSTAR = True

SmartSeeds3D(BaseDir = Data_dir, NPZfilename = NPZ_filename, model_name = Model_Name, model_dir = Model_dir, n_patches_per_image = n_patches_per_image,GenerateNPZ = False, TrainUNET = TrainUNET, TrainSTAR = TrainSTAR, PatchX= PatchX, PatchY= PatchY, PatchZ = PatchZ,  use_gpu = use_gpu_opencl,  batch_size = batch_size, depth = NetworkDepth, kern_size = Kernel, startfilter = startfilter, n_rays = Rays, epochs = Epochs, learning_rate = LearningRate)

Instance segmentation masks: 24
Semantic segmentation masks: 24
   24 raw images x    1 transformations   =    24 images
   24 images     x   16 patches per image =   384 patches in total
Input data:
/content/drive/My Drive/data/: target='BinaryMask/', sources=['Raw/'], axes='ZYX', pattern='*.tif*'
Transformations:
1 x Identity
Patch size:
16 x 128 x 128


100%|██████████| 24/24 [00:14<00:00,  1.70it/s]


Saving data to /content/drive/My Drive/data/VolumeSeg.npz.
Training UNET model
number of training images:	 346
number of validation images:	 38
image size (3D):		 (16, 128, 128)
axes:				 SZYXC
channels in / out:		 1 / 1
Config(axes='ZYXC', n_channel_in=1, n_channel_out=1, n_dim=3, probabilistic=False, train_batch_size=1, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_epochs=100, train_learning_rate=0.0001, train_loss='mse', train_reduce_lr={'patience': 5, 'factor': 0.5}, train_steps_per_epoch=400, train_tensorboard=True, unet_input_shape=(None, None, None, 1), unet_kern_size=3, unet_last_activation='linear', unet_n_depth=3, unet_n_first=48, unet_residual=True)


base_model.py (148): output path for model already exists, files may be overwritten: /content/drive/My Drive/data/UNETVolumeSeg


Epoch 1/100
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x7f4b5098e170>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x7f4b5098e170>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x7f4b4e68c320>: found multiple de